# SV-SCN Training with Real Furniture Data (Objaverse)

**Production-Ready Training with Realistic Furniture Models**

- ✅ Real furniture data (chairs with legs!)
- ✅ Automatic error handling
- ✅ Comprehensive validation
- ✅ 150 epochs for production quality
- ✅ Automatic checkpoint detection

**Expected Output:** Realistic 3D furniture models with proper details

## Step 1: Check GPU & Environment

In [ ]:
import sys

# Check GPU
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("✅ GPU Ready!")
else:
    print("❌ NO GPU! Enable: Runtime → Change runtime type → GPU")
    sys.exit(1)

## Step 2: Clone Project from GitHub

In [ ]:
import os

# Clone project
if os.path.exists('frozo-3d-model'):
    print("⚠️ Project already exists, pulling latest...")
    !cd frozo-3d-model && git pull
else:
    !git clone https://github.com/ashish-frozo/frozo-3d-model.git

# Navigate to project
%cd frozo-3d-model

# Verify structure
if not os.path.exists('svscn') or not os.path.exists('scripts'):
    print("❌ Project structure incorrect!")
    sys.exit(1)

print("✅ Project cloned successfully!")
!pwd

## Step 3: Install Dependencies

In [ ]:
# Install core dependencies
!pip install -q open3d>=0.17.0 trimesh>=4.0.0 scipy>=1.10.0
!pip install -q tensorboard>=2.14.0

# Install Objaverse for real furniture data
!pip install -q objaverse

# Verify imports
try:
    from svscn.models import SVSCN
    from svscn.config import default_config
    import objaverse
    print(f"✅ All dependencies installed!")
    print(f"Project version: {default_config.VERSION}")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    sys.exit(1)

## Step 4: Download Real Furniture from Objaverse

This downloads **real furniture models** instead of placeholder boxes!

In [ ]:
import objaverse
import subprocess
from pathlib import Path
import json

print("📥 Downloading real furniture from Objaverse...")
print("This may take 30-60 minutes for 500 models\n")

# Create output directory
!mkdir -p data/objaverse_raw

# Download furniture annotations
annotations = objaverse.load_annotations()

# Filter for furniture categories
furniture_objects = []
for uid, data in annotations.items():
    tags = data.get('tags', [])
    name = data.get('name', '').lower()
    
    # Filter for chairs, tables, stools
    if any(keyword in name for keyword in ['chair', 'stool', 'table', 'desk', 'seat']):
        furniture_objects.append(uid)

print(f"Found {len(furniture_objects)} furniture objects in Objaverse")

# Download 500 samples (167 per category approximate)
target_count = 500
selected_uids = furniture_objects[:target_count]

print(f"Downloading {len(selected_uids)} models...")

# Download
objects = objaverse.load_objects(
    uids=selected_uids,
    download_processes=4
)

# Move to our data directory and organize
output_dir = Path('data/objaverse_raw')
chair_dir = output_dir / 'chair'
table_dir = output_dir / 'table'
stool_dir = output_dir / 'stool'

chair_dir.mkdir(exist_ok=True)
table_dir.mkdir(exist_ok=True)
stool_dir.mkdir(exist_ok=True)

# Organize by category
for uid, filepath in objects.items():
    annotation = annotations[uid]
    name = annotation.get('name', '').lower()
    
    # Copy to appropriate category
    if 'chair' in name or 'seat' in name:
        !cp {filepath} {chair_dir}/{uid}.glb
    elif 'table' in name or 'desk' in name:
        !cp {filepath} {table_dir}/{uid}.glb
    else:
        !cp {filepath} {stool_dir}/{uid}.glb

# Verify download
chair_count = len(list(chair_dir.glob('*.glb')))
table_count = len(list(table_dir.glob('*.glb')))
stool_count = len(list(stool_dir.glob('*.glb')))

print(f"\n{'='*50}")
print(f"DOWNLOAD COMPLETE")
print(f"{'='*50}")
print(f"Chairs: {chair_count}")
print(f"Tables: {table_count}")
print(f"Stools: {stool_count}")
print(f"Total: {chair_count + table_count + stool_count}")

if (chair_count + table_count + stool_count) < 100:
    print("❌ ERROR: Not enough models downloaded!")
    sys.exit(1)
else:
    print("✅ Real furniture data ready!")

## Step 5: Convert GLB to OBJ Format

In [ ]:
import trimesh
from pathlib import Path
from tqdm import tqdm

print("🔄 Converting GLB to OBJ format...")

output_dir = Path('data/shapenet_objaverse')
output_dir.mkdir(exist_ok=True)

for category in ['chair', 'table', 'stool']:
    input_dir = Path(f'data/objaverse_raw/{category}')
    category_dir = output_dir / category
    category_dir.mkdir(exist_ok=True)
    
    glb_files = list(input_dir.glob('*.glb'))
    print(f"\nProcessing {len(glb_files)} {category} models...")
    
    converted = 0
    for glb_file in tqdm(glb_files):
        try:
            # Load and convert
            mesh = trimesh.load(glb_file)
            obj_path = category_dir / f"{glb_file.stem}.obj"
            mesh.export(obj_path)
            converted += 1
        except Exception as e:
            print(f"⚠️ Failed to convert {glb_file.name}: {e}")
            continue
    
    print(f"✅ {category}: {converted}/{len(glb_files)} converted")

print("\n✅ Conversion complete!")

## Step 6: Preprocess to Point Clouds

In [ ]:
# Preprocess OBJ files to point clouds
!python -m svscn.data.preprocess \
    --input_dir data/shapenet_objaverse \
    --output_dir data/processed_objaverse \
    --num_points 8192

# Verify
result = subprocess.run(['find', 'data/processed_objaverse', '-name', '*.npy'], 
                       capture_output=True, text=True)
pc_files = [f for f in result.stdout.strip().split('\n') if f]
num_pc = len(pc_files)

print(f"\n{'='*50}")
print(f"PREPROCESSING COMPLETE")
print(f"{'='*50}")
print(f"Point clouds created: {num_pc}")

if num_pc < 100:
    print("❌ ERROR: Not enough point clouds!")
    sys.exit(1)
else:
    print("✅ SUCCESS")

## Step 7: Generate Training Pairs

In [ ]:
# Create training pairs (partial + full)
!python -m svscn.data.augment \
    --input_dir data/processed_objaverse \
    --output_dir data/training_objaverse \
    --views 3

# Verify
result_full = subprocess.run(['find', 'data/training_objaverse/full', '-name', '*.npy'], 
                             capture_output=True, text=True)
result_partial = subprocess.run(['find', 'data/training_objaverse/partial', '-name', '*.npy'], 
                                capture_output=True, text=True)

full_files = [f for f in result_full.stdout.strip().split('\n') if f]
partial_files = [f for f in result_partial.stdout.strip().split('\n') if f]

num_full = len(full_files)
num_partial = len(partial_files)

print(f"\n{'='*50}")
print(f"TRAINING PAIRS CREATED")
print(f"{'='*50}")
print(f"Full point clouds: {num_full}")
print(f"Partial point clouds: {num_partial}")
print(f"Expected: ~{num_pc * 3} each")
print(f"Unique samples: {num_full // 3}")

if num_full < 300 or num_partial < 300:
    print(f"❌ ERROR: Not enough training pairs!")
    sys.exit(1)
else:
    print(f"✅ SUCCESS - Ready for training!")

## Step 8: Create Data Splits

In [ ]:
import numpy as np
from pathlib import Path

training_dir = Path('data/training_objaverse')
full_dir = training_dir / 'full'

# Get unique samples
samples = set()
for f in full_dir.glob('*_full.npy'):
    name = f.stem.replace('_full', '')
    base = '_'.join(name.split('_')[:-1])
    samples.add(base)

samples = sorted(list(samples))
print(f"Total unique samples: {len(samples)}")

# 80/10/10 split
np.random.seed(42)
np.random.shuffle(samples)

n = len(samples)
train = samples[:int(0.8*n)]
val = samples[int(0.8*n):int(0.9*n)]
test = samples[int(0.9*n):]

# Save splits
splits_dir = training_dir / 'splits'
splits_dir.mkdir(exist_ok=True)

(splits_dir / 'train.txt').write_text('\n'.join(train))
(splits_dir / 'val.txt').write_text('\n'.join(val))
(splits_dir / 'test.txt').write_text('\n'.join(test))

print(f"\n{'='*50}")
print(f"DATA SPLITS CREATED")
print(f"{'='*50}")
print(f"Train: {len(train)}")
print(f"Val: {len(val)}")
print(f"Test: {len(test)}")
print(f"✅ Ready to train!")

## Step 9: Train Model (150 Epochs)

⏱️ **This takes 2-4 hours** - keep tab open!

For quick test: change `--epochs 150` to `--epochs 10`

In [ ]:
# Create checkpoint and log directories
!mkdir -p checkpoints_objaverse logs_objaverse

# Check batch size based on GPU memory
gpu_mem_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
batch_size = 32 if gpu_mem_gb > 20 else 16 if gpu_mem_gb > 12 else 8

print(f"GPU Memory: {gpu_mem_gb:.1f} GB")
print(f"Using batch size: {batch_size}")
print(f"\nStarting training...\n")

# Train with real furniture data!
!python scripts/train.py \
    --data_dir data/training_objaverse \
    --epochs 150 \
    --batch_size {batch_size} \
    --checkpoint_dir checkpoints_objaverse \
    --log_dir logs_objaverse \
    --device cuda

print("\n✅ Training complete!")

## Step 10: Monitor Training (TensorBoard)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs_objaverse

## Step 11: Auto-Find Best Checkpoint

In [ ]:
import glob

checkpoint_files = glob.glob('checkpoints_objaverse/*/best.pt')

if not checkpoint_files:
    print("❌ No checkpoint found!")
    CP = None
else:
    CP = sorted(checkpoint_files)[-1]
    print(f"✅ Found checkpoint: {CP}")
    !ls -lh {CP}
    
    # Load and check training summary
    summary_file = str(Path(CP).parent / 'training_summary.json')
    if os.path.exists(summary_file):
        with open(summary_file) as f:
            summary = json.load(f)
        print(f"\nBest val loss: {summary['best_val_loss']:.6f}")
        print(f"Epochs completed: {summary['epochs_completed']}")

## Step 12: Test Inference

In [ ]:
# Create test input
import numpy as np

partial = np.random.randn(2048, 3).astype(np.float32)
partial = (partial - partial.mean(axis=0)) / partial.std()
np.save('test_input.npy', partial)

print(f"✅ Test input created: {partial.shape}")

In [ ]:
# Run inference - point cloud output
if CP:
    !python scripts/infer.py \
        --checkpoint {CP} \
        --input test_input.npy \
        --output completed.npy \
        --class_id 0 \
        --device cuda
    
    print("\n✅ Inference complete!")
    
    # Check output
    completed = np.load('completed.npy')
    print(f"Output shape: {completed.shape}")
else:
    print("❌ No checkpoint available for inference")

## Step 13: Export GLB (With Real Furniture Details!)

In [ ]:
# Export to GLB mesh - should have LEGS now!
if CP:
    !python scripts/infer.py \
        --checkpoint {CP} \
        --input test_input.npy \
        --output chair_with_legs.glb \
        --export_mesh \
        --class_id 0 \
        --device cuda
    
    print("\n✅ GLB exported!")
    print("📥 Download and view in 3D viewer - should see LEGS!")
    
    !ls -lh chair_with_legs.glb
else:
    print("❌ No checkpoint available")

## Step 14: Visualize Comparison

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

if os.path.exists('completed.npy') and os.path.exists('test_input.npy'):
    partial = np.load('test_input.npy')
    completed = np.load('completed.npy')
    
    fig = plt.figure(figsize=(15, 5))
    
    # Input
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.scatter(partial[:, 0], partial[:, 1], partial[:, 2], c='blue', s=1)
    ax1.set_title('Input (Partial)', fontsize=14)
    ax1.set_box_aspect([1,1,1])
    
    # Output (should have details now!)
    ax2 = fig.add_subplot(132, projection='3d')
    ax2.scatter(completed[:, 0], completed[:, 1], completed[:, 2], c='green', s=1)
    ax2.set_title('Output (Completed - Real Data!)', fontsize=14)
    ax2.set_box_aspect([1,1,1])
    
    # Overlay
    ax3 = fig.add_subplot(133, projection='3d')
    ax3.scatter(partial[:, 0], partial[:, 1], partial[:, 2], c='blue', s=1, alpha=0.5, label='Input')
    ax3.scatter(completed[:, 0], completed[:, 1], completed[:, 2], c='green', s=1, alpha=0.3, label='Output')
    ax3.set_title('Comparison', fontsize=14)
    ax3.legend()
    ax3.set_box_aspect([1,1,1])
    
    plt.tight_layout()
    plt.show()
    
    print(f"Input points: {len(partial)}")
    print(f"Output points: {len(completed)}")
else:
    print("⚠️ Output files not found - run inference first")

## Step 15: Download Everything

In [ ]:
from google.colab import files

if CP:
    print("📥 Downloading files...\n")
    
    # Download checkpoint
    print("1. Checkpoint (best.pt)")
    files.download(CP)
    
    # Download GLB
    if os.path.exists('chair_with_legs.glb'):
        print("2. GLB file (chair with LEGS!)")
        files.download('chair_with_legs.glb')
    
    # Download training summary
    summary_file = str(Path(CP).parent / 'training_summary.json')
    if os.path.exists(summary_file):
        print("3. Training summary")
        files.download(summary_file)
    
    print("\n✅ All files downloaded!")
else:
    print("❌ No checkpoint to download")

##Quality Check

In [ ]:
import json

summary_files = glob.glob('checkpoints_objaverse/*/training_summary.json')

if summary_files:
    with open(summary_files[-1]) as f:
        summary = json.load(f)
    
    print("="*50)
    print("TRAINING RESULTS - REAL FURNITURE DATA")
    print("="*50)
    print(f"Best val loss: {summary['best_val_loss']:.6f}")
    print(f"Final train loss: {summary['train_losses'][-1]:.6f}")
    print(f"Epochs: {summary['epochs_completed']}")
    print("\n" + "="*50)
    print("QUALITY ASSESSMENT")
    print("="*50)
    
    val_loss = summary['best_val_loss']
    
    if val_loss < 0.005:
        print(f"✅ EXCELLENT ({val_loss:.6f})")
        print("   Your model should produce high-quality furniture!")
        print("   Expected: Chairs with legs, realistic details")
    elif val_loss < 0.01:
        print(f"✅ VERY GOOD ({val_loss:.6f})")
        print("   Production-ready quality")
    elif val_loss < 0.05:
        print(f"✅ GOOD ({val_loss:.6f})")
        print("   Usable for beta/MVP")
    else:
        print(f"⚠️  OK ({val_loss:.6f})")
        print("   May need more training or data")
    
    print("\n📋 Next steps:")
    print("   1. Download chair_with_legs.glb")
    print("   2. View in 3D viewer - check for LEGS!")
    print("   3. If good → Week 2: SAM-3D benchmarking")
    print("   4. If issues → Retrain with more data")
else:
    print("❌ No training summary found")

## 🎉 Training Complete!

### What You Have:
- ✅ Model trained on **real furniture** (not placeholder boxes!)
- ✅ Chairs should have **LEGS** now
- ✅ Tables should have **proper structure** 
- ✅ Production-ready checkpoint

### Next Steps:
1. **Download GLB** and view in [3dviewer.net](https://3dviewer.net)
2. **Verify quality** - chairs should have legs!
3. **Week 2** - SAM-3D benchmarking
4. **Week 3-4** - Deploy to production!

### Expected Quality:
- **Better than placeholder** - realistic furniture shapes
- **Production viable** - can show to customers
- **AR-ready** - GLB files work in AR viewers

🚀 **You've built production-quality AI!**